# Booking Dataset Cleaning

## Purpose
This notebook performs **deterministic cleaning and schema standardization** of the raw Booking.com dataset.
The goal is to produce a **stable listing-level dataset** that can be reliably used in all downstream stages
(exploration, enrichment, labeling, and modeling).

---

## Cleaning Principles
- **One row = one listing (`hotel_id`)** — no aggregation or deduplication.
- **No imputation** — missing values are preserved and handled downstream.
- **Normalize, don’t over-process** — fix invalid values without altering semantics.
- **EDA-driven decisions** — all non-trivial choices are motivated by prior exploration.

---

## What This Notebook Does
- Drops columns with no usable signal.
- Flattens and validates geographic coordinates (`lat`, `lon`).
- Normalizes strings (trim, empty → null).
- Cleans nested arrays/structs and removes empty elements.
- Validates numeric ranges and enforces consistent types.
- Canonicalizes ambiguous fields (e.g. room capacity → `max_guests`).

---

## Output
The cleaned dataset is saved to:
dbfs:/tmp/booking_clean/booking_clean.parquet


In [0]:
import os
from pyspark.sql import functions as F

storage_account = "lab94290"  
acct = storage_account

def set_sas_for_account(acct: str, sas_token: str):
    sas_token = sas_token.lstrip('?')
    spark.conf.set(f"fs.azure.account.auth.type.{acct}.dfs.core.windows.net", "SAS")
    spark.conf.set(f"fs.azure.sas.token.provider.type.{acct}.dfs.core.windows.net",
                   "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
    spark.conf.set(f"fs.azure.sas.fixed.token.{acct}.dfs.core.windows.net", sas_token)

# ============================================================
# INSERT SAS TOKEN HERE (provided by course staff)
# ============================================================
booking_sas_token = os.getenv("BOOKING_SAS_TOKEN", "<INSERT_SAS_TOKEN_HERE>")

if booking_sas_token == "<INSERT_SAS_TOKEN_HERE>" or booking_sas_token.strip() == "":
    raise ValueError("Missing SAS token. Paste it into booking_sas_token or set env var BOOKING_SAS_TOKEN.")


set_sas_for_account(acct, booking_sas_token)

booking_container = "booking"
booking_fname = "booking_1_9.parquet"
booking_path = f"abfss://{booking_container}@{acct}.dfs.core.windows.net/{booking_fname}"

booking_df = spark.read.parquet(booking_path)
display(booking_df.limit(5))

availability city coordinates country description fine_print hotel_id house_rules images location managed_by manager_language_spoken manager_score metro_railway_access most_popular_facilities number_of_reviews popular_facilities property_highlights property_information property_surroundings review_score reviews_scores title top_reviews url List(List(1 king bed, null, 2, One-Bedroom Apartment)) Vrindāvan List(27.5737379354334, 77.6546632927292) India Featuring a garden, Mero vrindavan Luxury aavas features accommodations in Vrindāvan. This property offers access to a terrace, free private parking, and free Wifi.

This apartment will provide guests with 1 bedroom, a flat-screen TV, and air conditioning. The apartment also includes 1 bathroom.

Bharatpur Train Station is 31 miles from the apartment, while Mathura Train Station is 9 miles from the property. Agra Airport is 41 miles away. This property does not accommodate bachelor(ette) or similar parties. 12919785 List(List(From 12:00 PM to 11:00 PM You need to let the property know what time you'll be arriving in advance., Check-in), List(From 8:00 AM to 11:00 AM, Check-out), List(Cancellation and prepayment policies vary according to accommodation type. Enter your stay dates and check the conditions of your selected option., Cancellation/ prepayment), List(Child policies Children of all ages are welcome. To see correct prices and occupancy info, add the number and ages of children in your group to your search. Crib and extra bed policies Cribs and extra beds aren't available at this property., Children & Beds), List(There's no age requirement for check-in, No age restriction), List(Smoking is not allowed., Smoking), List(Parties/events are not allowed, Parties), List(Free! Pets are allowed. No extra charges., Pets)) List(https://cf.bstatic.com/xdata/images/hotel/max200/603653035.jpg?k=389f38be85031d2743f99b2fa6dfb9057557c739a921c74399ef5a0ef2d13edd&o=, https://cf.bstatic.com/xdata/images/hotel/max200/603653144.jpg?k=f363b0b79f3a6cd34c0deea79fe914db2e9ff4de19718324c9984c473167f21c&o=, https://cf.bstatic.com/xdata/images/hotel/max200/603653114.jpg?k=d1db0a18879b81d88ced90184f7baf3d69a655690cb52845fcc66c4734ab1135&o=, https://cf.bstatic.com/xdata/images/hotel/max200/603653103.jpg?k=b4196fc9b98950240db78d9648f8c8f88cedad19a0421c8c2c09a056bda0b214&o=, https://cf.bstatic.com/xdata/images/hotel/max200/603653086.jpg?k=c947befcb2f56d906e33488fe3bf63dc93a7902d037e2a8e36b8f87dc782261b&o=, https://cf.bstatic.com/xdata/images/hotel/max200/603653112.jpg?k=5e4ac49a4cd1ebfceb59314e1c81e80b115250e34a7f8f2784998ea924831857&o=, https://cf.bstatic.com/xdata/images/hotel/max200/603653083.jpg?k=ea5c5f07e22411dbe5a8efe3b38d2186907f87c4d6a7b59e4225902920d7e57b&o=, https://cf.bstatic.com/xdata/images/hotel/max200/603653080.jpg?k=1a8ad31134e2089042228673eb427c58b040904707917123325d70b8e70bbeb4&o=, https://cf.bstatic.com/xdata/images/hotel/max200/603653068.jpg?k=e398c11b34fa5dc518938f033b9234c55b024a0857fa0ff7954399ed1968975f&o=, https://cf.bstatic.com/xdata/images/hotel/max200/603653075.jpg?k=d64617957b2a3a2aa8038da09e33c7502bad000c06fea6f2580d4d4ec61931d6&o=, https://cf.bstatic.com/xdata/images/hotel/max200/603653077.jpg?k=6e8a8110d71315753c82807a0983eff2cd53c2073abd818f8cec72e646fb2d87&o=, https://cf.bstatic.com/xdata/images/hotel/max200/603653088.jpg?k=b2410bbebbf7e7747d63e1eb371676d2b7c142886e49d764bb2eca8ff22ad1b4&o=, https://cf.bstatic.com/xdata/images/hotel/max200/603653093.jpg?k=01e8307caea449129657723dd6674cc021f583ec1df15e1bd56aac26ddbee099&o=, https://cf.bstatic.com/xdata/images/hotel/max200/603653098.jpg?k=2941324ca574eccb9e3d0eea87e5d46d8151d66b6753ff4fcefd4cdb53203ff7&o=, https://cf.bstatic.com/xdata/images/hotel/max200/603653107.jpg?k=2030818f224ee99b97e139e3f0bc79778dd21f32204ae3618fdb13ae7dbac145&o=, https://cf.bstatic.com/xdata/images/hotel/max200/603653119.jpg?k=c46bd7884d30ab675cb06330515457f6d56f47bae3ffbc616e449913cae680f6&o=, https://cf.bstatic.com/xdata/images/hotel/max200/603653

In [0]:
booking_n = booking_df.count()
print("Booking rows:", booking_n)

# Explicit alias for clarity in later steps
booking_raw_df = booking_df

Booking rows: 3239391


In [0]:
from pyspark.sql import functions as F

DROP_COLS = ["popular_facilities", "reviews_scores", "managed_by"]

# Flatten coordinates struct into two numeric columns
booking_df = (
    booking_df
    .drop(*[c for c in DROP_COLS if c in booking_df.columns])
    .withColumn("lat", F.col("coordinates.lan").cast("double"))
    .withColumn("lon", F.col("coordinates.lon").cast("double"))
    .drop("coordinates")
)

display(booking_df.select("hotel_id", "url", "lat", "lon").limit(5))


hotel_id,url,lat,lon
12919785,https://www.booking.com/hotel/in/mero-vrindavan-luxury-aavas.html,27.5737379354334,77.6546632927292
11851686,https://www.booking.com/hotel/fr/appartement-centre-ville-nice4.html,43.7104008,7.2551083
10017137,https://www.booking.com/hotel/kz/apartamenty-2-kom-v-zhk-al-tair-vozle-giper-dina.html?chal_t=1756121381660&force_referer=,50.281956966893,57.149165545824
12955753,https://www.booking.com/hotel/us/estrella-mountain-masterpiece.html,33.343518,-112.440015
10116229,https://www.booking.com/hotel/gr/cycladic-house-for-6-people.html,36.353748895964,25.470865750698


In [0]:
from pyspark.sql import functions as F

def _trim_or_null(col):
    """Trim string; if empty after trim -> null."""
    return F.when(F.length(F.trim(col)) == 0, F.lit(None)).otherwise(F.trim(col))


In [0]:
booking_df = booking_df.withColumn(
    "manager_language_spoken",
    F.array_distinct(
        F.filter(
            F.transform(F.col("manager_language_spoken"), lambda x: _trim_or_null(x)),
            lambda x: x.isNotNull()
        )
    )
)

In [0]:
booking_df = booking_df.withColumn(
    "most_popular_facilities",
    F.array_distinct(
        F.filter(
            F.transform(F.col("most_popular_facilities"), lambda x: _trim_or_null(x)),
            lambda x: x.isNotNull()
        )
    )
)


In [0]:
booking_df = booking_df.withColumn(
    "availability",
    F.transform(
        F.col("availability"),
        lambda a: F.struct(
            _trim_or_null(a["bed_configuration"]).alias("bed_configuration"),
            _trim_or_null(a["room_type"]).alias("room_type"),
            F.coalesce(
                a["max_number_of_guests"].cast("long"),
                F.regexp_extract(a["max_number_of guests"], r"\d+", 0).cast("long")
            ).alias("max_guests"),
        )
    )
)


In [0]:
booking_df = booking_df.withColumn(
    "house_rules",
    F.filter(
        F.transform(
            F.col("house_rules"),
            lambda r: F.struct(
                _trim_or_null(r["rule"]).alias("rule"),
                _trim_or_null(r["description"]).alias("description"),
            )
        ),
        lambda r: r["rule"].isNotNull() | r["description"].isNotNull()
    )
)

In [0]:
booking_df = booking_df.withColumn(
    "property_surroundings",
    F.filter(
        F.transform(
            F.col("property_surroundings"),
            lambda p: F.struct(
                p["distance"].cast("double").alias("distance"),
                _trim_or_null(p["distance_unit"]).alias("distance_unit"),
                _trim_or_null(p["location_name"]).alias("location_name"),
            )
        ),
        lambda p: p["location_name"].isNotNull()
    )
)

In [0]:
booking_df = booking_df.withColumn(
    "top_reviews",
    F.filter(
        F.transform(
            F.col("top_reviews"),
            lambda t: F.struct(
                _trim_or_null(t["location"]).alias("location"),
                _trim_or_null(t["reviewer_name"]).alias("reviewer_name"),
                _trim_or_null(t["review"]).alias("review"),
            )
        ),
        lambda t: t["review"].isNotNull()
    )
)

In [0]:
display(
    booking_df.select(
        "hotel_id",
        F.size("most_popular_facilities").alias("n_facilities"),
        F.size("manager_language_spoken").alias("n_mgr_lang"),
        F.size("availability").alias("n_availability"),
        F.size("house_rules").alias("n_house_rules"),
        F.size("property_surroundings").alias("n_surroundings"),
        F.size("top_reviews").alias("n_top_reviews"),
    ).limit(5)
)


hotel_id,n_facilities,n_mgr_lang,n_availability,n_house_rules,n_surroundings,n_top_reviews
12919785,2,2,1,8,10,0
11851686,3,2,1,9,10,1
10017137,6,5,1,11,0,1
12955753,2,8,1,9,9,0
10116229,7,2,6,10,10,5


In [0]:
from pyspark.sql import functions as F

def trim_empty_to_null(c):
    return F.when(F.length(F.trim(F.col(c))) == 0, F.lit(None)).otherwise(F.trim(F.col(c)))

# Flat string columns
STRING_COLS = [
    "city",
    "country",
    "description",
    "fine_print",
    "hotel_id",
    "location",
    "property_highlights",
    "property_information",
    "title",
    "url",
]

# Apply only to columns that actually exist
for c in [x for x in STRING_COLS if x in booking_df.columns]:
    booking_df = booking_df.withColumn(c, trim_empty_to_null(c))

# Quick check
display(
    booking_df.select(
        "hotel_id", "url", "title", "city", "country", "location"
    ).limit(5)
)

hotel_id,url,title,city,country,location
12919785,https://www.booking.com/hotel/in/mero-vrindavan-luxury-aavas.html,Mero vrindavan Luxury aavas,Vrindāvan,India,"Omaxe enternity Vrindavan, 281121 Vrindāvan, India"
11851686,https://www.booking.com/hotel/fr/appartement-centre-ville-nice4.html,Appartement centre ville,Nice,France,"8 rue cro de capeu, 06000 Nice, France"
10017137,https://www.booking.com/hotel/kz/apartamenty-2-kom-v-zhk-al-tair-vozle-giper-dina.html?chal_t=1756121381660&force_referer=,Апартаменты 2 ком в ЖК Альтаир возле гипер Дина,Aktobe,Kazakhstan,"40 проспект Санкибай Батыра, 030000 Aktobe, Kazakhstan"
12955753,https://www.booking.com/hotel/us/estrella-mountain-masterpiece.html,Estrella Mountain Masterpiece,Liberty,US,"Liberty, 85338, United States"
10116229,https://www.booking.com/hotel/gr/cycladic-house-for-6-people.html,Pleasure Black Beach Perissa Santorini,Perissa,Greece,"Perissa Santorini, Perissa, 84703, Greece"


In [0]:
from pyspark.sql import functions as F

# 1) Enforce types 
booking_df = (
    booking_df
    .withColumn("number_of_reviews", F.col("number_of_reviews").cast("bigint"))
    .withColumn("review_score",      F.col("review_score").cast("double"))
    .withColumn("manager_score",     F.col("manager_score").cast("double"))
    .withColumn("metro_railway_access", F.col("metro_railway_access").cast("boolean"))
)

# 2) Range checks (invalidate impossible values -> null)
booking_df = (
    booking_df
    .withColumn(
        "review_score",
        F.when((F.col("review_score") >= 0) & (F.col("review_score") <= 10), F.col("review_score"))
         .otherwise(F.lit(None))
    )
    .withColumn(
        "manager_score",
        F.when((F.col("manager_score") >= 0) & (F.col("manager_score") <= 10), F.col("manager_score"))
         .otherwise(F.lit(None))
    )
    .withColumn(
        "number_of_reviews",
        F.when(F.col("number_of_reviews") >= 0, F.col("number_of_reviews"))
         .otherwise(F.lit(None))
    )
)

# 3) Numeric summary (min/max/mean + null counts)
numeric_cols = ["review_score", "manager_score", "number_of_reviews"]

summary_exprs = []
for c in numeric_cols:
    summary_exprs += [
        F.count(F.col(c)).alias(f"{c}__non_null"),
        F.sum(F.col(c).isNull().cast("int")).alias(f"{c}__nulls"),
        F.min(F.col(c)).alias(f"{c}__min"),
        F.expr(f"percentile_approx({c}, 0.5)").alias(f"{c}__median"),
        F.max(F.col(c)).alias(f"{c}__max"),
        F.avg(F.col(c)).alias(f"{c}__mean"),
    ]

display(booking_df.agg(*summary_exprs))

# 4) Quick peek
display(booking_df.select("hotel_id", *numeric_cols, "metro_railway_access").limit(10))


review_score__non_null,review_score__nulls,review_score__min,review_score__median,review_score__max,review_score__mean,manager_score__non_null,manager_score__nulls,manager_score__min,manager_score__median,manager_score__max,manager_score__mean,number_of_reviews__non_null,number_of_reviews__nulls,number_of_reviews__min,number_of_reviews__median,number_of_reviews__max,number_of_reviews__mean
1548130,1691261,0.0,8.8,10.0,7.776112794145303,1183670,2055721,2.5,9.0,10.0,8.930874483598382,1548130,1691261,0,15,44439,115.14621511113408


hotel_id,review_score,manager_score,number_of_reviews,metro_railway_access
12919785,9.5,7.0,2,true
11851686,5.0,null,17,true
10017137,7.4,null,22,true
12955753,0.0,8.4,0,true
10116229,8.8,7.9,15,false
9805533,0.0,8.5,0,true
171062,8.5,null,292,true
11107900,0.0,9.2,0,false
11620921,0.0,9.2,0,false
6402511,8.0,8.5,2,false


In [0]:
FINAL_COL_ORDER = [
    # Identifiers
    "hotel_id",
    "url",
    "title",

    # Location
    "city",
    "country",
    "location",
    "lat",
    "lon",
    "metro_railway_access",

    # Descriptions & policies
    "description",
    "fine_print",
    "property_highlights",
    "property_information",

    # Facilities & rules
    "most_popular_facilities",
    "house_rules",
    "manager_language_spoken",

    # Availability / rooms
    "availability",

    # Reviews & scores
    "review_score",
    "manager_score",
    "number_of_reviews",
    "top_reviews",

    # Context
    "property_surroundings",

    # Media
    "images",
]

booking_df = booking_df.select(*FINAL_COL_ORDER)
booking_df.printSchema()

root
 |-- hotel_id: string (nullable = true)
 |-- url: string (nullable = true)
 |-- title: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- location: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- metro_railway_access: boolean (nullable = true)
 |-- description: string (nullable = true)
 |-- fine_print: string (nullable = true)
 |-- property_highlights: string (nullable = true)
 |-- property_information: string (nullable = true)
 |-- most_popular_facilities: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- house_rules: array (nullable = true)
 |    |-- element: struct (containsNull = false)
 |    |    |-- rule: string (nullable = true)
 |    |    |-- description: string (nullable = true)
 |-- manager_language_spoken: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- availability: array (nullable = true)
 |    |-- element: s

In [0]:
health = booking_df.select(
    F.count("*").alias("rows"),
    F.mean(F.col("hotel_id").isNull().cast("int")).alias("hotel_id_null_rate"),
    F.mean(F.col("lat").isNull().cast("int")).alias("lat_null_rate"),
    F.mean(F.col("lon").isNull().cast("int")).alias("lon_null_rate"),
    F.mean((F.size("most_popular_facilities") == 0).cast("int")).alias("facilities_empty_rate"),
    F.mean((F.size("manager_language_spoken") == 0).cast("int")).alias("mgr_lang_empty_rate"),
    F.mean((F.size("availability") == 0).cast("int")).alias("availability_empty_rate"),
    F.mean((F.size("house_rules") == 0).cast("int")).alias("house_rules_empty_rate"),
    F.mean((F.size("property_surroundings") == 0).cast("int")).alias("surroundings_empty_rate"),
    F.mean((F.size("top_reviews") == 0).cast("int")).alias("top_reviews_empty_rate"),
)
display(health)

rows,hotel_id_null_rate,lat_null_rate,lon_null_rate,facilities_empty_rate,mgr_lang_empty_rate,availability_empty_rate,house_rules_empty_rate,surroundings_empty_rate,top_reviews_empty_rate
3239391,0.0,9.545003983773493E-4,9.545003983773493E-4,0.01704363567102582,0.0,0.017596208670086447,0.0037831184935686987,0.07894045516580123,0.38639701104312507


In [0]:
BOOKING_CLEAN_PATH = "dbfs:/tmp/booking_clean/booking_clean.parquet"

booking_df.write.mode("overwrite").parquet(BOOKING_CLEAN_PATH)

print("Saved cleaned Booking dataset to:", BOOKING_CLEAN_PATH)
print("Final row count:", booking_df.count())


Saved cleaned Booking dataset to: dbfs:/tmp/booking_clean/booking_clean.parquet
Final row count: 3239391
